In [4]:
import os
import pandas as pd
import numpy as np
import pyiat
from scipy import stats
data=pd.read_excel('merged_iat_all.xlsx')
data['correct']=1-data['error']


In [5]:
data=data[data['subj_idx']!=106]
data

,Unnamed: 0,subj_idx,stim4,stim3,stim1,block,condition,error,response,rt,...,AUC,xpos_flips,ypos_flips,xpos_reversals,ypos_reversals,RT,initiation_time,idle_time,newblock,correct
0,id0001,101,NaN,unreal,unreal,1,real/unreal,0,2,8875,...,0.058596,11,5,3,1,8860,1844,5546,1,1
1,id0002,101,NaN,real,real,1,real/unreal,0,1,1922,...,0.737511,4,2,2,0,1890,218,640,1,1
2,id0003,101,NaN,unreal,unreal,1,real/unreal,0,2,1875,...,0.238636,2,0,1,0,1860,500,734,1,1
3,id0004,101,NaN,real,real,1,real/unreal,0,1,2125,...,1.283048,7,6,3,0,2109,78,360,1,1
4,id0005,101,NaN,real,real,1,real/unreal,0,1,1578,...,0.064626,0,0,0,0,1562,406,579,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31855,id5196,160,NaN,unreal,unreal,7,compatible,0,2,907,...,0.770872,2,0,2,0,891,31,422,4,1
31856,id5197,160,NaN,unreal,unreal,7,compatible,1,1,1640,...,0.659926,2,3,2,0,1625,0,424,4,0
31857,id5198,160,NaN,unreal,unreal,7,compatible,0,2,1062,...,1.033698,1,0,1,0,1047,16,407,4,1
31858,id5199,160,NaN,real,real,7,compatible,0,1,1141,...,0.116451,0,0,0,0,1109,0,483,4,1


# calculating the weighted d_scores

In [6]:
d=data[(data['block']==4)|(data['block']==7)]

In [7]:
print(stats.pearsonr(d['rt'], d['MAD']))
print(stats.pearsonr(d['rt'], d['AD']))
print(stats.pearsonr(d['rt'], d['AUC']))
print(stats.pearsonr(d['rt'], d['MAD']))

(0.20835209546335606, 6.852385500633556e-226)
(0.24947808814376604, 0.0)
(0.1047906034917141, 1.1896478485475777e-57)
(0.20835209546335606, 6.852385500633556e-226)


In [8]:
error=d.groupby(['subj_idx'])['error'].mean()
error=pd.DataFrame(error)
print(error.mean())
error=d.groupby(['subj_idx','condition'])['error'].mean()
error=pd.DataFrame(error)
print(error['error'][:,'compatible'].mean())
print(error['error'][:,'incompatible'].mean())

error    0.145776
dtype: float64
0.04448275862068965
0.2470689655172414


In [9]:
rtmean=d.groupby(['subj_idx','stim3','condition'])['rt'].mean()
rtstd=d.groupby(['subj_idx','stim3'])['rt'].std()
rtmean=pd.DataFrame(rtmean)
rtstd=pd.DataFrame(rtstd)


In [10]:
dscore_ev1=[]
for i in range(60):
    if i==5:
        continue
    if i==51:
        continue
  
    dscore_ev1.append((rtmean['rt'][i+101,'ev1','compatible']-rtmean['rt'][i+101,'ev1','incompatible'])/rtstd['rt'][i+101,'ev1'])
print(np.mean(dscore_ev1))
print(np.std(dscore_ev1))

-0.36993261572913677
0.36216270917578103


In [11]:
dscore_ev2=[]
for i in range(60):
    if i==5:
        continue
    if i==51:
        continue
    dscore_ev2.append((rtmean['rt'][i+101,'ev2','compatible']-rtmean['rt'][i+101,'ev2','incompatible'])/rtstd['rt'][i+101,'ev2'])
print(np.mean(dscore_ev2))
print(np.std(dscore_ev2))

-0.3217326697069881
0.4476427912497744


In [12]:
rtmean_overall=d.groupby(['subj_idx','condition'])['rt'].mean()
rtstd_overall=d.groupby(['subj_idx'])['rt'].std()
rtmean_overall=pd.DataFrame(rtmean_overall)
rtstd_overall=pd.DataFrame(rtstd_overall)

In [13]:
dscore=[]
for i in range(60):
    if i==5:
        continue
    if i==51:
        continue    
    dscore.append((rtmean_overall['rt'][i+101,'compatible']-rtmean_overall['rt'][i+101,'incompatible'])/rtstd_overall['rt'][i+101])
print(np.mean(dscore))
print(np.std(dscore))

-0.2884352242038331
0.32821297157911083


In [14]:
print(stats.ttest_rel(dscore_ev1,dscore_ev2))

Ttest_relResult(statistic=-0.9499532198601762, pvalue=0.34614661179651796)


In [15]:
rtmean_modality=d.groupby(['subj_idx','stim4','condition'])['rt'].mean()
rtstd_modality=d.groupby(['subj_idx','stim4'])['rt'].std()
rtmean_modality=pd.DataFrame(rtmean_modality)
rtstd_modality=pd.DataFrame(rtstd_modality)
rtmean_modality

rt
subj_idx stim4 condition            
101      imag  compatible    1697.60
               incompatible  2329.24
         word  compatible    1832.44
               incompatible  3470.06
102      imag  compatible    1362.46
...                              ...
159      word  incompatible  2377.48
160      imag  compatible    1065.58
               incompatible  1372.60
         word  compatible    1082.26
               incompatible  1424.40

[232 rows x 1 columns]

In [16]:
dscore_word=[]
for i in range(60):
    if i==5:
        continue
    if i==51:
        continue
    dscore_word.append((rtmean_modality['rt'][i+101,'word','compatible']-rtmean_modality['rt'][i+101,'word','incompatible'])/rtstd_modality['rt'][i+101,'word'])
print(np.mean(dscore_word))
print(np.std(dscore_word))

-0.33514834402646976
0.39061023241493636


In [17]:
dscore_imag=[]
for i in range(60):
    if i==5:
        continue
    if i==51:
        continue
    dscore_imag.append((rtmean_modality['rt'][i+101,'imag','compatible']-rtmean_modality['rt'][i+101,'imag','incompatible'])/rtstd_modality['rt'][i+101,'imag'])
print(np.mean(dscore_imag))
print(np.std(dscore_imag))

-0.38064915975783814
0.40546826356638


In [18]:
stats.ttest_rel(dscore_word,dscore_imag)


Ttest_relResult(statistic=0.9998355991928034, pvalue=0.32161580928918837)

In [19]:
stats.ttest_rel(dscore_word,[0 for i in range(58)])

Ttest_relResult(statistic=-6.477850037392553, pvalue=2.3464971090713906e-08)

In [20]:
stats.ttest_rel(dscore_imag,[0 for i in range(58)])

Ttest_relResult(statistic=-7.087701781686106, pvalue=2.272725659398343e-09)

# RT

In [21]:
data1=data[(data.block==4)|(data.block==7)]

data1=data1[data1['correct']==1]
data1=data1[data1['rt']<5000]

In [22]:
RT =pd.DataFrame(columns=('subj_idx','total_com','total_incom','ev1_com','ev1_incom','ev2_com','ev2_incom','ev1_imag_com','ev1_imag_incom','ev1_word_com','ev1_word_incom','ev2_imag_com','ev2_imag_com','ev2_word_com','ev2_word_com'))
RT.subj_idx=range(101,161)
for i in range(101,161):
    
    RT.iloc[i-101,1]=data1[(data1.subj_idx==i)&(data1.condition=='compatible')].RT.mean()
    RT.iloc[i-101,2]=data1[(data1.subj_idx==i)&(data1.condition=='incompatible')].RT.mean()
    RT.iloc[i-101,3]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.condition=='compatible')].RT.mean()
    RT.iloc[i-101,4]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.condition=='incompatible')].RT.mean()
    RT.iloc[i-101,5]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.condition=='compatible')].RT.mean()
    RT.iloc[i-101,6]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.condition=='incompatible')].RT.mean()
    RT.iloc[i-101,7]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='imag')&(data1.condition=='compatible')].RT.mean()
    RT.iloc[i-101,8]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='imag')&(data1.condition=='incompatible')].RT.mean()
    RT.iloc[i-101,9]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='word')&(data1.condition=='compatible')].RT.mean()
    RT.iloc[i-101,10]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='word')&(data1.condition=='incompatible')].RT.mean()
    RT.iloc[i-101,11]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='imag')&(data1.condition=='compatible')].RT.mean()
    RT.iloc[i-101,12]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='imag')&(data1.condition=='incompatible')].RT.mean()
    RT.iloc[i-101,13]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='word')&(data1.condition=='compatible')].RT.mean()
    RT.iloc[i-101,14]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='word')&(data1.condition=='incompatible')].RT.mean()

RT=RT[(RT['subj_idx']!=106)&(RT['subj_idx']!=152)]

RT

,subj_idx,total_com,total_incom,ev1_com,ev1_incom,ev2_com,ev2_incom,ev1_imag_com,ev1_imag_incom,ev1_word_com,ev1_word_incom,ev2_imag_com,ev2_imag_com,ev2_word_com,ev2_word_com
0,101,1728.04,2661.32,1561.52,2328.94,1743.88,NaN,1436.2,NaN,1686.84,2328.94,1689.42,NaN,1798.33,NaN
1,102,1378.09,1548.31,1478.09,1587.53,1258.72,1535.86,1474.94,1508.35,1482.14,1691.08,1238.76,1565.17,1278.68,1513.88
2,103,1697.3,1679.84,1913.59,1975.39,1487.12,1538.89,1812.36,2077.75,2099.17,1958.33,1395.68,1267.57,1578.56,1810.21
3,104,1284.14,1246.4,1362.46,1278.33,1198.76,1217.04,1254.88,1250.04,1470.04,1441,1163.28,1169.83,1235.71,1264.26
4,105,1103,1191.3,1118.92,1266.18,1132.22,1195.33,1075.64,996,1164,1326.22,1114.4,1096.33,1150.04,1294.33
6,107,1364.33,1226.54,1264.88,1301.98,1438.1,1319.55,1158.44,1353.59,1371.32,1245.2,1288.08,1288.7,1594.38,1349.12
7,108,1698.78,1974.6,1699.4,1939.31,1641.44,1747.68,1423.16,1751.7,1975.64,2056.56,1492,1683.27,1828.25,1836.25
8,109,1339.69,1417.16,1290.04,1496.38,1186.22,1649.58,1417.08,1592,1163,1428.88,1160.56,1571.92,1211.88,1716.14
9,110,1108.76,1405.22,1039.32,1311.27,1094.15,NaN,1035.04,1422.5,1043.6,1302,980.72,NaN,1217.43,NaN
10,111,1178.67,1551.42,1264.16,1583.27,1183.29,1520,1058.24,1276.41,1470.08,2162.89,1036.36,1279.89,1336.33,1824.13


In [23]:
print(RT.total_com.mean())
print(RT.total_com.std())
print(RT.total_incom.mean())
print(RT.total_incom.std())

1279.717621963506
252.43765174283195
1456.040393054651
347.2111013626752


In [24]:
print(stats.ttest_rel(RT.total_com,RT.total_incom))


print(stats.ttest_rel(RT.loc[:,['ev1_com','ev1_incom']].dropna().ev1_com,RT.loc[:,['ev1_com','ev1_incom']].dropna().ev1_incom))
print(stats.ttest_rel(RT.loc[:,['ev2_com','ev2_incom']].dropna().ev2_com,RT.loc[:,['ev2_com','ev2_incom']].dropna().ev2_incom))
print(stats.ttest_rel(RT.loc[:,['ev1_com','ev2_com']].dropna().ev1_com,RT.loc[:,['ev1_com','ev2_com']].dropna().ev2_com))
print(stats.ttest_rel(RT.loc[:,['ev1_incom','ev2_incom']].dropna().ev1_incom,RT.loc[:,['ev1_incom','ev2_incom']].dropna().ev2_incom))

Ttest_relResult(statistic=-5.962752949885333, pvalue=1.6552008164348018e-07)
Ttest_relResult(statistic=-8.305545520328424, pvalue=2.4355875501153184e-11)
Ttest_relResult(statistic=-4.846175919985628, pvalue=1.3083530414255042e-05)
Ttest_relResult(statistic=1.908606490408155, pvalue=0.06144319324811225)
Ttest_relResult(statistic=1.3684072152360112, pvalue=0.1774265595457057)


# AUC


In [25]:
AUC =pd.DataFrame(columns=('subj_idx','total_com','total_incom','ev1_com','ev1_incom','ev2_com','ev2_incom','ev1_imag_com','ev1_imag_incom','ev1_word_com','ev1_word_incom','ev2_imag_com','ev2_imag_com','ev2_word_com','ev2_word_com'))
AUC.subj_idx=range(101,161)
for i in range(101,161):
    
    AUC.iloc[i-101,1]=data1[(data1.subj_idx==i)&(data1.condition=='compatible')].AUC.mean()
    AUC.iloc[i-101,2]=data1[(data1.subj_idx==i)&(data1.condition=='incompatible')].AUC.mean()
    AUC.iloc[i-101,3]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.condition=='compatible')].AUC.mean()
    AUC.iloc[i-101,4]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.condition=='incompatible')].AUC.mean()
    AUC.iloc[i-101,5]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.condition=='compatible')].AUC.mean()
    AUC.iloc[i-101,6]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.condition=='incompatible')].AUC.mean()
    AUC.iloc[i-101,7]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='imag')&(data1.condition=='compatible')].AUC.mean()
    AUC.iloc[i-101,8]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='imag')&(data1.condition=='incompatible')].AUC.mean()
    AUC.iloc[i-101,9]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='word')&(data1.condition=='compatible')].AUC.mean()
    AUC.iloc[i-101,10]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='word')&(data1.condition=='incompatible')].AUC.mean()
    AUC.iloc[i-101,11]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='imag')&(data1.condition=='compatible')].AUC.mean()
    AUC.iloc[i-101,12]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='imag')&(data1.condition=='incompatible')].AUC.mean()
    AUC.iloc[i-101,13]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='word')&(data1.condition=='compatible')].AUC.mean()
    AUC.iloc[i-101,14]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='word')&(data1.condition=='incompatible')].AUC.mean()

AUC=AUC[(AUC['subj_idx']!=106)&(AUC['subj_idx']!=152)]

AUC

,subj_idx,total_com,total_incom,ev1_com,ev1_incom,ev2_com,ev2_incom,ev1_imag_com,ev1_imag_incom,ev1_word_com,ev1_word_incom,ev2_imag_com,ev2_imag_com,ev2_word_com,ev2_word_com
0,101,0.220054,0.309531,0.179924,0.160095,0.118338,NaN,0.198956,NaN,0.160892,0.160095,0.090208,NaN,0.146468,NaN
1,102,0.291277,0.340261,0.370665,0.288731,0.17659,0.290236,0.289804,0.237109,0.474629,0.356237,0.139115,0.261894,0.214064,0.311492
2,103,0.177698,0.203211,0.451994,0.169593,0.0652707,0.0627108,0.575072,0.617232,0.226351,0.0949862,0.0152883,0.00896517,0.115253,0.116456
3,104,0.0178585,0.05416,0.0285775,0.0747254,-0.0435874,-0.0104991,0.00552685,0.0497203,0.0516282,0.218505,-0.0491682,-0.0113877,-0.0377741,-0.00961056
4,105,0.267616,0.311087,0.251615,0.343882,0.174779,0.154676,0.199923,0.224526,0.305461,0.370406,0.152872,0.0664537,0.196686,0.242898
6,107,0.400146,0.431087,0.232354,0.289261,0.532465,0.674271,0.1633,0.0777974,0.301408,0.521871,0.436413,0.602107,0.632519,0.743428
7,108,0.274658,0.352167,0.116041,0.358619,0.399767,0.570445,0.0563546,0.245807,0.175728,0.429126,0.341672,0.547929,0.472385,0.601405
8,109,0.190847,0.204441,0.101403,0.248051,0.128358,0.315286,0.0443161,0.154064,0.158489,0.314395,0.0610244,0.33901,0.195691,0.29495
9,110,0.36653,0.481398,0.227253,0.438517,0.416728,NaN,0.265007,0.0331475,0.189499,0.472297,0.306736,NaN,0.536285,NaN
10,111,0.410947,0.380599,0.422158,0.416097,0.420679,0.274988,0.388646,0.381749,0.45567,0.480976,0.437979,0.229017,0.402658,0.333217


In [26]:
print(AUC.total_com.mean())
print(AUC.total_com.std())
print(AUC.total_incom.mean())
print(AUC.total_incom.std())

0.2880683048009633
0.11410257235937805
0.3157285655696449
0.1014317013185887


In [27]:
#配对t-test
print(stats.ttest_rel(AUC.total_com,AUC.total_incom))


print(stats.ttest_rel(AUC.loc[:,['ev1_com','ev1_incom']].dropna().ev1_com,AUC.loc[:,['ev1_com','ev1_incom']].dropna().ev1_incom))
print(stats.ttest_rel(AUC.loc[:,['ev2_com','ev2_incom']].dropna().ev2_com,AUC.loc[:,['ev2_com','ev2_incom']].dropna().ev2_incom))
print(stats.ttest_rel(AUC.loc[:,['ev1_com','ev2_com']].dropna().ev1_com,AUC.loc[:,['ev1_com','ev2_com']].dropna().ev2_com))
print(stats.ttest_rel(AUC.loc[:,['ev1_incom','ev2_incom']].dropna().ev1_incom,AUC.loc[:,['ev1_incom','ev2_incom']].dropna().ev2_incom))

Ttest_relResult(statistic=-2.602391486231143, pvalue=0.01177775667762345)
Ttest_relResult(statistic=-1.7369088317687424, pvalue=0.08790138934447812)
Ttest_relResult(statistic=0.10349090127096341, pvalue=0.9179957878306066)
Ttest_relResult(statistic=0.44877762199903387, pvalue=0.6553239333391443)
Ttest_relResult(statistic=1.7729090305905095, pvalue=0.08245975718968791)


# MAD

In [28]:
MAD =pd.DataFrame(columns=('subj_idx','total_com','total_incom','ev1_com','ev1_incom','ev2_com','ev2_incom','ev1_imag_com','ev1_imag_incom','ev1_word_com','ev1_word_incom','ev2_imag_com','ev2_imag_com','ev2_word_com','ev2_word_com'))
MAD.subj_idx=range(101,161)
for i in range(101,161):

    MAD.iloc[i-101,1]=data1[(data1.subj_idx==i)&(data1.condition=='compatible')].MAD.mean()
    MAD.iloc[i-101,2]=data1[(data1.subj_idx==i)&(data1.condition=='incompatible')].MAD.mean()
    MAD.iloc[i-101,3]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.condition=='compatible')].MAD.mean()
    MAD.iloc[i-101,4]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.condition=='incompatible')].MAD.mean()
    MAD.iloc[i-101,5]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.condition=='compatible')].MAD.mean()
    MAD.iloc[i-101,6]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.condition=='incompatible')].MAD.mean()
    MAD.iloc[i-101,7]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='imag')&(data1.condition=='compatible')].MAD.mean()
    MAD.iloc[i-101,8]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='imag')&(data1.condition=='incompatible')].MAD.mean()
    MAD.iloc[i-101,9]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='word')&(data1.condition=='compatible')].MAD.mean()
    MAD.iloc[i-101,10]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='word')&(data1.condition=='incompatible')].MAD.mean()
    MAD.iloc[i-101,11]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='imag')&(data1.condition=='compatible')].MAD.mean()
    MAD.iloc[i-101,12]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='imag')&(data1.condition=='incompatible')].MAD.mean()
    MAD.iloc[i-101,13]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='word')&(data1.condition=='compatible')].MAD.mean()
    MAD.iloc[i-101,14]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='word')&(data1.condition=='incompatible')].MAD.mean()

MAD=MAD[(MAD['subj_idx']!=106)&(MAD['subj_idx']!=152)]
MAD


,subj_idx,total_com,total_incom,ev1_com,ev1_incom,ev2_com,ev2_incom,ev1_imag_com,ev1_imag_incom,ev1_word_com,ev1_word_incom,ev2_imag_com,ev2_imag_com,ev2_word_com,ev2_word_com
0,101,0.295763,0.4935,0.245345,0.234565,0.165782,NaN,0.235555,NaN,0.255134,0.234565,0.130349,NaN,0.201215,NaN
1,102,0.442288,0.552117,0.544765,0.421017,0.210283,0.488177,0.477879,0.354359,0.630763,0.508187,0.174691,0.400394,0.245876,0.554015
2,103,0.319498,0.338285,0.688497,0.34102,0.121601,0.143365,0.736662,0.939068,0.600195,0.241345,0.023066,-0.000798385,0.220136,0.287528
3,104,0.0214991,0.0754108,0.0326974,0.0902693,-0.0657905,-0.0275854,-0.00724464,0.0547071,0.0726395,0.294752,-0.0806096,-0.0343588,-0.0503538,-0.020812
4,105,0.352712,0.451692,0.304643,0.561128,0.244972,0.197041,0.235487,0.245122,0.376681,0.631352,0.202958,0.0683599,0.286986,0.325722
6,107,0.616397,0.666115,0.47999,0.625405,0.734064,0.773735,0.338924,0.614542,0.621056,0.637354,0.550869,0.730724,0.924891,0.814955
7,108,0.38938,0.528094,0.191937,0.583918,0.5331,0.792674,0.0719883,0.603335,0.311887,0.571782,0.435065,0.70413,0.655644,0.914422
8,109,0.297511,0.297223,0.211964,0.359771,0.166043,0.379994,0.125418,0.336044,0.29851,0.376519,0.0699514,0.381847,0.262135,0.378406
9,110,0.519144,0.892597,0.30599,0.790629,0.589396,NaN,0.340893,0.776554,0.271087,0.791802,0.401904,NaN,0.793191,NaN
10,111,0.659168,0.670542,0.736918,0.705592,0.584006,0.489512,0.593379,0.597994,0.880456,0.908832,0.51088,0.317912,0.660179,0.706873


In [29]:
mad_mean=MAD.mean()
mad_mean

subj_idx          130.551724
total_com           0.445650
total_incom         0.512816
ev1_com             0.430785
ev1_incom           0.505498
ev2_com             0.397368
ev2_incom           0.475197
ev1_imag_com        0.374457
ev1_imag_incom      0.570473
ev1_word_com        0.487031
ev1_word_incom      0.480295
ev2_imag_com        0.324878
ev2_imag_com        0.422317
ev2_word_com        0.473537
ev2_word_com        0.527615
dtype: float64

In [30]:
mad_std=MAD.std()
mad_std

subj_idx          17.235390
total_com          0.198785
total_incom        0.189019
ev1_com            0.230872
ev1_incom          0.237406
ev2_com            0.257935
ev2_incom          0.305603
ev1_imag_com       0.245243
ev1_imag_incom     0.339901
ev1_word_com       0.248978
ev1_word_incom     0.243517
ev2_imag_com       0.249935
ev2_imag_com       0.330467
ev2_word_com       0.286316
ev2_word_com       0.336262
dtype: float64

In [31]:
print(stats.ttest_rel(MAD.total_com,MAD.total_incom))

Ttest_relResult(statistic=-3.4534264006895787, pvalue=0.001050687351100404)


In [32]:
#配对t-test


print(stats.ttest_rel(MAD.loc[:,['ev1_com','ev1_incom']].dropna().ev1_com,MAD.loc[:,['ev1_com','ev1_incom']].dropna().ev1_incom))
print(stats.ttest_rel(MAD.loc[:,['ev2_com','ev2_incom']].dropna().ev2_com,MAD.loc[:,['ev2_com','ev2_incom']].dropna().ev2_incom))
print(stats.ttest_rel(MAD.loc[:,['ev1_com','ev2_com']].dropna().ev1_com,MAD.loc[:,['ev1_com','ev2_com']].dropna().ev2_com))
print(stats.ttest_rel(MAD.loc[:,['ev1_incom','ev2_incom']].dropna().ev1_incom,MAD.loc[:,['ev1_incom','ev2_incom']].dropna().ev2_incom))


Ttest_relResult(statistic=-2.5669027859757585, pvalue=0.012960762568766158)
Ttest_relResult(statistic=-1.6035479303996547, pvalue=0.1152409716838085)
Ttest_relResult(statistic=1.0078566539878053, pvalue=0.3178584051269785)
Ttest_relResult(statistic=1.1110196310526887, pvalue=0.2719819474770452)


# AD

In [33]:
AD =pd.DataFrame(columns=('subj_idx','total_com','total_incom','ev1_com','ev1_incom','ev2_com','ev2_incom','ev1_imag_com','ev1_imag_incom','ev1_word_com','ev1_word_incom','ev2_imag_com','ev2_imag_com','ev2_word_com','ev2_word_com'))
AD.subj_idx=range(101,161)
for i in range(101,161):

    AD.iloc[i-101,1]=data1[(data1.subj_idx==i)&(data1.condition=='compatible')].AD.mean()
    AD.iloc[i-101,2]=data1[(data1.subj_idx==i)&(data1.condition=='incompatible')].AD.mean()
    AD.iloc[i-101,3]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.condition=='compatible')].AD.mean()
    AD.iloc[i-101,4]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.condition=='incompatible')].AD.mean()
    AD.iloc[i-101,5]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.condition=='compatible')].AD.mean()
    AD.iloc[i-101,6]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.condition=='incompatible')].AD.mean()
    AD.iloc[i-101,7]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='imag')&(data1.condition=='compatible')].AD.mean()
    AD.iloc[i-101,8]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='imag')&(data1.condition=='incompatible')].AD.mean()
    AD.iloc[i-101,9]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='word')&(data1.condition=='compatible')].AD.mean()
    AD.iloc[i-101,10]=data1[(data1.subj_idx==i)&(data1.stim3=='ev1')&(data1.stim4=='word')&(data1.condition=='incompatible')].AD.mean()
    AD.iloc[i-101,11]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='imag')&(data1.condition=='compatible')].AD.mean()
    AD.iloc[i-101,12]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='imag')&(data1.condition=='incompatible')].AD.mean()
    AD.iloc[i-101,13]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='word')&(data1.condition=='compatible')].AD.mean()
    AD.iloc[i-101,14]=data1[(data1.subj_idx==i)&(data1.stim3=='ev2')&(data1.stim4=='word')&(data1.condition=='incompatible')].AD.mean()

AD=AD[(AD['subj_idx']!=106)&(AD['subj_idx']!=152)]
AD

,subj_idx,total_com,total_incom,ev1_com,ev1_incom,ev2_com,ev2_incom,ev1_imag_com,ev1_imag_incom,ev1_word_com,ev1_word_incom,ev2_imag_com,ev2_imag_com,ev2_word_com,ev2_word_com
0,101,0.0678292,0.133111,0.0684779,0.0587929,0.0392659,NaN,0.0534652,NaN,0.0834906,0.0587929,0.0353379,NaN,0.0431938,NaN
1,102,0.119965,0.149696,0.151253,0.103841,0.0511678,0.137284,0.120019,0.0953092,0.191412,0.114998,0.0443268,0.153108,0.0580089,0.125417
2,103,0.0868453,0.1009,0.210666,0.113407,0.0285631,0.0467574,0.240155,0.361653,0.156603,0.072033,0.00487005,0.00869588,0.0522562,0.084819
3,104,0.00738387,0.020532,0.00640371,0.023766,-0.0117519,-0.00286803,-0.00151912,0.0108061,0.0143265,0.0982855,-0.0135114,-0.00169572,-0.00991913,-0.00404034
4,105,0.0746991,0.104966,0.0567862,0.135985,0.0496382,0.0507525,0.0415598,0.060963,0.0726471,0.152657,0.0385355,0.00814873,0.0607409,0.0933562
6,107,0.139101,0.135232,0.102633,0.137491,0.160677,0.125862,0.0490879,0.137645,0.156179,0.137321,0.0755976,0.121485,0.249301,0.130055
7,108,0.138885,0.209301,0.0629373,0.226401,0.188518,0.332141,0.0213729,0.155996,0.104502,0.270405,0.148001,0.292983,0.239166,0.385983
8,109,0.0726349,0.0630293,0.0603797,0.0775931,0.0301315,0.0829611,0.0577577,0.0751482,0.0630017,0.0793189,0.013832,0.0788044,0.0464311,0.086524
9,110,0.149091,0.258835,0.0859177,0.213431,0.168777,NaN,0.0950921,0.105083,0.0767434,0.22246,0.0998524,NaN,0.243695,NaN
10,111,0.194096,0.205401,0.232779,0.213674,0.159464,0.156892,0.167238,0.171652,0.29832,0.293049,0.135484,0.103446,0.184444,0.22459


In [34]:
AD.mean()


subj_idx          130.551724
total_com           0.124966
total_incom         0.146894
ev1_com             0.120179
ev1_incom           0.144721
ev2_com             0.110201
ev2_incom           0.139026
ev1_imag_com        0.097554
ev1_imag_incom      0.159475
ev1_word_com        0.142918
ev1_word_incom      0.139742
ev2_imag_com        0.087450
ev2_imag_com        0.116204
ev2_word_com        0.134066
ev2_word_com        0.159570
dtype: float64

In [35]:
AD.std()

subj_idx          17.235390
total_com          0.061012
total_incom        0.061004
ev1_com            0.073500
ev1_incom          0.077275
ev2_com            0.076719
ev2_incom          0.109895
ev1_imag_com       0.070632
ev1_imag_incom     0.108266
ev1_word_com       0.089342
ev1_word_incom     0.081710
ev2_imag_com       0.072896
ev2_imag_com       0.107452
ev2_word_com       0.088674
ev2_word_com       0.124384
dtype: float64

In [36]:
print(stats.ttest_rel(AD.total_com,AD.total_incom))
print(stats.ttest_rel(AD.loc[:,['ev1_com','ev1_incom']].dropna().ev1_com,AD.loc[:,['ev1_com','ev1_incom']].dropna().ev1_incom))
print(stats.ttest_rel(AD.loc[:,['ev2_com','ev2_incom']].dropna().ev2_com,AD.loc[:,['ev2_com','ev2_incom']].dropna().ev2_incom))
print(stats.ttest_rel(AD.loc[:,['ev1_com','ev2_com']].dropna().ev1_com,AD.loc[:,['ev1_com','ev2_com']].dropna().ev2_com))
print(stats.ttest_rel(AD.loc[:,['ev1_incom','ev2_incom']].dropna().ev1_incom,AD.loc[:,['ev1_incom','ev2_incom']].dropna().ev2_incom))


Ttest_relResult(statistic=-3.739532227076708, pvalue=0.0004295977017574891)
Ttest_relResult(statistic=-2.61595731402315, pvalue=0.011415282610263042)
Ttest_relResult(statistic=-1.818110031968445, pvalue=0.07516300432719823)
Ttest_relResult(statistic=0.9328894273021644, pvalue=0.35488198451633224)
Ttest_relResult(statistic=0.6611261711135594, pvalue=0.5116281492404)


# linear mixed effect models

In [37]:
from scipy import stats
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd

data1

,Unnamed: 0,subj_idx,stim4,stim3,stim1,block,condition,error,response,rt,...,AUC,xpos_flips,ypos_flips,xpos_reversals,ypos_reversals,RT,initiation_time,idle_time,newblock,correct
2640,0,101,NaN,real,real,4,compatible,0,1,3141,...,-0.051148,0,2,0,0,3125,921,2374,4,1
2641,1,101,word,ev1,ev1_word,4,compatible,0,1,2297,...,0.047250,1,0,0,0,2281,875,1671,4,1
2642,2,101,word,ev2,ev2_word,4,compatible,0,2,2453,...,-0.062450,0,0,0,0,2422,1453,2031,4,1
2643,3,101,imag,ev2,ev2_imag,4,compatible,0,2,1219,...,0.183083,2,2,0,0,1203,578,751,4,1
2644,4,101,word,ev1,ev1_word,4,compatible,0,1,1110,...,0.299585,3,0,1,0,1094,0,593,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31854,id5195,160,NaN,real,real,7,compatible,0,1,984,...,0.509905,5,0,3,0,968,0,203,4,1
31855,id5196,160,NaN,unreal,unreal,7,compatible,0,2,907,...,0.770872,2,0,2,0,891,31,422,4,1
31857,id5198,160,NaN,unreal,unreal,7,compatible,0,2,1062,...,1.033698,1,0,1,0,1047,16,407,4,1
31858,id5199,160,NaN,real,real,7,compatible,0,1,1141,...,0.116451,0,0,0,0,1109,0,483,4,1


In [49]:
data1['group']=0
for sub in range(101,134):
    data1['group'][data1['subj_idx']==sub]=1
for sub in range(134,161):
    data1['group'][data1['subj_idx']==sub]=2
    

/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

In [52]:
data1

,Unnamed: 0,subj_idx,stim4,stim3,stim1,block,condition,error,response,rt,...,ypos_flips,xpos_reversals,ypos_reversals,RT,initiation_time,idle_time,newblock,correct,group,g
2640,0,101,NaN,real,real,4,compatible,0,1,3141,...,2,0,0,3125,921,2374,4,1,1,1
2641,1,101,word,ev1,ev1_word,4,compatible,0,1,2297,...,0,0,0,2281,875,1671,4,1,1,1
2642,2,101,word,ev2,ev2_word,4,compatible,0,2,2453,...,0,0,0,2422,1453,2031,4,1,1,1
2643,3,101,imag,ev2,ev2_imag,4,compatible,0,2,1219,...,2,0,0,1203,578,751,4,1,1,1
2644,4,101,word,ev1,ev1_word,4,compatible,0,1,1110,...,0,1,0,1094,0,593,4,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31854,id5195,160,NaN,real,real,7,compatible,0,1,984,...,0,3,0,968,0,203,4,1,2,2
31855,id5196,160,NaN,unreal,unreal,7,compatible,0,2,907,...,0,2,0,891,31,422,4,1,2,2
31857,id5198,160,NaN,unreal,unreal,7,compatible,0,2,1062,...,0,1,0,1047,16,407,4,1,2,2
31858,id5199,160,NaN,real,real,7,compatible,0,1,1141,...,0,0,0,1109,0,483,4,1,2,2


In [54]:
mixed = smf.mixedlm("RT~g+condition+g*condition+(1|subj_idx)",data1,groups=data1['subj_idx'])

mixed_fit = mixed.fit()
#print the summary
print(mixed_fit.summary())

                    Mixed Linear Model Regression Results
Model:                   MixedLM       Dependent Variable:       RT          
No. Observations:        19590         Method:                   REML        
No. Groups:              58            Scale:                    289092.6177 
Min. group size:         214           Log-Likelihood:           -151071.5906
Max. group size:         388           Converged:                Yes         
Mean group size:         337.8                                               
-----------------------------------------------------------------------------
                              Coef.   Std.Err.   z    P>|z|  [0.025   0.975] 
-----------------------------------------------------------------------------
Intercept                    2041.621  370.036  5.517 0.000 1316.364 2766.877
condition[T.incompatible]    -118.863   23.759 -5.003 0.000 -165.430  -72.297
g                             103.444  137.909  0.750 0.453 -166.853  373.741
g:cond

In [39]:
data1

,Unnamed: 0,subj_idx,stim4,stim3,stim1,block,condition,error,response,rt,...,xpos_flips,ypos_flips,xpos_reversals,ypos_reversals,RT,initiation_time,idle_time,newblock,correct,group
2640,0,101,NaN,real,real,4,compatible,0,1,3141,...,0,2,0,0,3125,921,2374,4,1,1
2641,1,101,word,ev1,ev1_word,4,compatible,0,1,2297,...,1,0,0,0,2281,875,1671,4,1,1
2642,2,101,word,ev2,ev2_word,4,compatible,0,2,2453,...,0,0,0,0,2422,1453,2031,4,1,1
2643,3,101,imag,ev2,ev2_imag,4,compatible,0,2,1219,...,2,2,0,0,1203,578,751,4,1,1
2644,4,101,word,ev1,ev1_word,4,compatible,0,1,1110,...,3,0,1,0,1094,0,593,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31854,id5195,160,NaN,real,real,7,compatible,0,1,984,...,5,0,3,0,968,0,203,4,1,2
31855,id5196,160,NaN,unreal,unreal,7,compatible,0,2,907,...,2,0,2,0,891,31,422,4,1,2
31857,id5198,160,NaN,unreal,unreal,7,compatible,0,2,1062,...,1,0,1,0,1047,16,407,4,1,2
31858,id5199,160,NaN,real,real,7,compatible,0,1,1141,...,0,0,0,0,1109,0,483,4,1,2


In [40]:
(data1[data1['group']==1].rt.mean()-data1[data1['group']==2].rt.mean())/data1.rt.std()

0.0036946722815476554

In [41]:
(data1[data1['group']==1].MAD.mean()-data1[data1['group']==2].MAD.mean())/data1.MAD.std()

-0.11246948449563365

In [42]:
(data1[data1['group']==1].AD.mean()-data1[data1['group']==2].AD.mean())/data1.MAD.std()

-0.04392678900231829

In [43]:
data1['g']=data1['group']
data1

,Unnamed: 0,subj_idx,stim4,stim3,stim1,block,condition,error,response,rt,...,ypos_flips,xpos_reversals,ypos_reversals,RT,initiation_time,idle_time,newblock,correct,group,g
2640,0,101,NaN,real,real,4,compatible,0,1,3141,...,2,0,0,3125,921,2374,4,1,1,1
2641,1,101,word,ev1,ev1_word,4,compatible,0,1,2297,...,0,0,0,2281,875,1671,4,1,1,1
2642,2,101,word,ev2,ev2_word,4,compatible,0,2,2453,...,0,0,0,2422,1453,2031,4,1,1,1
2643,3,101,imag,ev2,ev2_imag,4,compatible,0,2,1219,...,2,0,0,1203,578,751,4,1,1,1
2644,4,101,word,ev1,ev1_word,4,compatible,0,1,1110,...,0,1,0,1094,0,593,4,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31854,id5195,160,NaN,real,real,7,compatible,0,1,984,...,0,3,0,968,0,203,4,1,2,2
31855,id5196,160,NaN,unreal,unreal,7,compatible,0,2,907,...,0,2,0,891,31,422,4,1,2,2
31857,id5198,160,NaN,unreal,unreal,7,compatible,0,2,1062,...,0,1,0,1047,16,407,4,1,2,2
31858,id5199,160,NaN,real,real,7,compatible,0,1,1141,...,0,0,0,1109,0,483,4,1,2,2


In [44]:
smf.mixedlm("rt ~ C(g) * C(condition)",
        data = data1).fit()

AttributeError: 'groups' is a required keyword argument in MixedLM.from_formula

In [ ]:
formula = 'RT~C(g)*C(condition)'

anova_inter = sm.stats.anova_lm(smf.ols('RT~C(stim3)*C(condition)', data = data1).fit())  # dc_sales_est 是一个模型对象
print(anova_inter)

                            df        sum_sq       mean_sq           F  \
C(stim3)                   3.0  1.768299e+07  5.894332e+06   16.444927   
C(condition)               1.0  1.085367e+08  1.085367e+08  302.812645   
C(stim3):C(condition)      3.0  1.002785e+07  3.342616e+06    9.325753   
Residual               19582.0  7.018748e+09  3.584286e+05         NaN   

                             PR(>F)  
C(stim3)               1.140504e-10  
C(condition)           2.579629e-67  
C(stim3):C(condition)  3.706505e-06  
Residual                        NaN  


In [ ]:
import pingouin


In [ ]:
pingouin.mixed_anova(dv='rt', between='group',within='condition', subject='subj_idx', data=data1,effsize='np2')

/Library/anaconda3/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.9, the latest is 0.5.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/Library/anaconda3/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,2329.310798,1,56,2329.310798,0.014361,9.050425e-01,0.000256,NaN
1,condition,901658.659681,1,56,901658.659681,42.407886,2.206631e-08,0.430940,1.0
2,Interaction,255817.029382,1,56,255817.029382,12.031892,1.012976e-03,0.176857,NaN


In [ ]:
pingouin.anova(data=data1, dv='rt', between=['group'], ss_type=2, detailed=True, effsize='np2')

,Source,SS,DF,MS,F,p-unc,np2
0,group,2.400760e+04,1,24007.599493,0.065704,0.797701,0.000003
1,Within,7.157299e+09,19588,365392.027246,NaN,NaN,NaN


In [ ]:
pingouin.anova(data=data1, dv='rt', between=['stim3','condition'], ss_type=2, detailed=False, effsize='np2')

,Source,SS,DF,MS,F,p-unc,np2
0,stim3,1.552625e+07,3.0,5.175417e+06,14.434435,2.170840e-09,0.002207
1,condition,1.084711e+08,1.0,1.084711e+08,302.530121,2.965998e-67,0.015214
2,stim3 * condition,1.004751e+07,3.0,3.349168e+06,9.340958,3.625799e-06,0.001429
3,Residual,7.021059e+09,19582.0,3.585466e+05,NaN,NaN,NaN


In [ ]:
formula = 'RT~C(stim3)*C(condition)'

anova_inter = sm.stats.anova_lm(smf.ols('RT~C(stim3)*C(condition)', data = data1).fit())  # dc_sales_est 是一个模型对象
print(anova_inter)

                            df        sum_sq       mean_sq           F  \
C(stim3)                   3.0  1.768299e+07  5.894332e+06   16.444927   
C(condition)               1.0  1.085367e+08  1.085367e+08  302.812645   
C(stim3):C(condition)      3.0  1.002785e+07  3.342616e+06    9.325753   
Residual               19582.0  7.018748e+09  3.584286e+05         NaN   

                             PR(>F)  
C(stim3)               1.140504e-10  
C(condition)           2.579629e-67  
C(stim3):C(condition)  3.706505e-06  
Residual                        NaN  


In [ ]:
pingouin.anova(data=data1, dv='rt', between=['condition','stim3','stim4'], ss_type=2, detailed=True, effsize='np2')

,Source,SS,DF,MS,F,p-unc,np2
0,condition,4.990750e+07,1.0,4.990750e+07,144.939920,4.103122e-33,0.016706
1,stim3,1.202421e+07,1.0,1.202421e+07,34.920360,3.566511e-09,0.004077
2,stim4,6.219252e+07,1.0,6.219252e+07,180.617708,9.215261e-41,0.020733
3,condition * stim3,5.928376e+06,1.0,5.928376e+06,17.217018,3.366738e-05,0.002014
4,condition * stim4,2.047335e+05,1.0,2.047335e+05,0.594581,4.406744e-01,0.000070
5,stim3 * stim4,5.836560e+05,1.0,5.836560e+05,1.695037,1.929735e-01,0.000199
6,condition * stim3 * stim4,8.928822e+04,1.0,8.928822e+04,0.259308,6.106090e-01,0.000030
7,Residual,2.937499e+09,8531.0,3.443323e+05,NaN,NaN,NaN


In [ ]:
#anova:RT~2(congruency)*2(ev1/ev2)*2(word/imag)####！
formula1 = 'rt~C(condition)*C(stim3)*C(stim4)'

anova1 = sm.stats.anova_lm(smf.ols(formula1,data = data1).fit())  
anova1

,df,sum_sq,mean_sq,F,PR(>F)
C(condition),1.0,5.760142e+07,5.760142e+07,167.284388,6.577771e-38
C(stim3),3.0,1.448469e+07,4.828229e+06,14.022004,4.081136e-09
C(stim4),1.0,6.424435e+07,6.424435e+07,186.576582,4.908966e-42
C(condition):C(stim3),3.0,6.535213e+06,2.178404e+06,6.326458,2.787340e-04
C(condition):C(stim4),1.0,2.049936e+05,2.049936e+05,0.595336,4.403843e-01
C(stim3):C(stim4),3.0,6.203942e+05,2.067981e+05,0.600577,6.145768e-01
C(condition):C(stim3):C(stim4),3.0,3.725008e+05,1.241669e+05,0.360602,7.814691e-01
Residual,8531.0,2.937499e+09,3.443323e+05,NaN,NaN


# visualize-mouse indices

In [ ]:
data2=data1[(data1['stim1'] != 'real')&(data1['stim1']!='unreal')]
data2.iloc[:,16:26]
data2

,Unnamed: 0,subj_idx,stim4,stim3,stim1,block,condition,error,response,rt,...,xpos_flips,ypos_flips,xpos_reversals,ypos_reversals,RT,initiation_time,idle_time,newblock,correct,group
2641,1,101,word,ev1,ev1_word,4,compatible,0,1,2297,...,1,0,0,0,2281,875,1671,4,1,1
2642,2,101,word,ev2,ev2_word,4,compatible,0,2,2453,...,0,0,0,0,2422,1453,2031,4,1,1
2643,3,101,imag,ev2,ev2_imag,4,compatible,0,2,1219,...,2,2,0,0,1203,578,751,4,1,1
2644,4,101,word,ev1,ev1_word,4,compatible,0,1,1110,...,3,0,1,0,1094,0,593,4,1,1
2652,12,101,imag,ev2,ev2_imag,4,compatible,0,2,1593,...,1,1,0,0,1578,953,1312,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31850,id5191,160,imag,ev2,ev2_imag,7,compatible,0,2,938,...,2,0,0,0,922,250,360,4,1,2
31851,id5192,160,imag,ev2,ev2_imag,7,compatible,0,2,953,...,2,1,2,0,938,250,407,4,1,2
31852,id5193,160,word,ev1,ev1_word,7,compatible,0,1,1047,...,3,2,3,0,1032,344,658,4,1,2
31853,id5194,160,word,ev1,ev1_word,7,compatible,0,1,1219,...,2,0,0,0,1203,94,375,4,1,2


In [ ]:
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(5,7),dpi=200);


data_com=data2[data2['condition']=='compatible']
a1=sns.barplot(x='stim3',y='RT',hue='stim4',data=data_com,capsize=0.05,errwidth=2,ci=50,palette='BuGn_r');


a1.tick_params(labelsize=7)
plt.ylim([1000,2000])
plt.yticks([1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000])

plt.ylim([1000,1850])



plt.show()
plt.savefig('fig3b_com.png')

/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [ ]:
plt.figure(figsize=(5,7),dpi=200);


data_incom=data2[data2['condition']=='incompatible']
a1=sns.barplot(x='stim3',y='RT',hue='stim4',data=data_incom,capsize=0.05,errwidth=2,ci=50,palette='Blues_r');


a1.tick_params(labelsize=7)
plt.ylim([1000,2000])
plt.yticks([1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000])

plt.ylim([1000,1850])



plt.show()
plt.savefig('fig3b_incom.png')


/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  app.launch_new_instance()


In [ ]:
group1=data1[data1['subj_idx']<134]
group1['group']=1
group2=data1[data1['subj_idx']>133]
group2['group']=2


/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
data3=pd.concat([group1, group2])

In [ ]:
formula1 = 'rt~C(condition)*C(stim3)*C(stim4)*C(group)'

anova1 = sm.stats.anova_lm(smf.ols(formula1,data = data3).fit())  
anova1

,df,sum_sq,mean_sq,F,PR(>F)
C(condition),1.0,5.760142e+07,5.760142e+07,168.884331,2.990207e-38
C(stim3),3.0,1.448469e+07,4.828229e+06,14.156114,3.356086e-09
C(stim4),1.0,6.424435e+07,6.424435e+07,188.361040,2.042900e-42
C(group),1.0,1.739279e+05,1.739279e+05,0.509947,4.751812e-01
C(condition):C(stim3),3.0,6.249666e+06,2.083222e+06,6.107897,3.805009e-04
C(condition):C(stim4),1.0,1.997132e+05,1.997132e+05,0.585548,4.441669e-01
C(stim3):C(stim4),3.0,6.086862e+05,2.028954e+05,0.594879,6.183020e-01
C(condition):C(group),1.0,2.494454e+07,2.494454e+07,73.136076,1.420447e-17
C(stim3):C(group),3.0,1.334750e+07,4.449166e+06,13.044719,1.696427e-08
C(stim4):C(group),1.0,4.112821e+05,4.112821e+05,1.205857,2.721852e-01


In [ ]:
pingouin.anova(data=data3, dv='rt', between=['stim3','condition','stim4','group'], ss_type=2, detailed=False, effsize='np2')

,Source,SS,DF,MS,F,p-unc,np2
0,stim3,1.059877e+07,1.0,1.059877e+07,31.075045,2.558161e-08,0.003633
1,condition,4.977646e+07,1.0,4.977646e+07,145.941947,2.500204e-33,0.016835
2,stim4,6.033031e+07,1.0,6.033031e+07,176.885269,5.795920e-40,0.020332
3,group,7.236478e+04,1.0,7.236478e+04,0.212170,6.450838e-01,0.000025
4,stim3 * condition,4.732396e+06,1.0,4.732396e+06,13.875134,1.966277e-04,0.001625
5,stim3 * stim4,5.592577e+05,1.0,5.592577e+05,1.639714,2.003996e-01,0.000192
6,condition * stim4,9.620369e+04,1.0,9.620369e+04,0.282064,5.953653e-01,0.000033
7,stim3 * group,1.216459e+05,1.0,1.216459e+05,0.356659,5.503833e-01,0.000042
8,condition * group,2.512599e+07,1.0,2.512599e+07,73.668080,1.087347e-17,0.008569
9,stim4 * group,2.739932e+05,1.0,2.739932e+05,0.803334,3.701239e-01,0.000094


# pic-word analysis

In [ ]:
RT =pd.DataFrame(columns=('subj_idx','total_com','total_incom','imag_com','imag_incom','word_com','word_incom'))
RT.subj_idx=range(101,161)
for i in range(101,161):
    
    RT.iloc[i-101,1]=data1[(data1.subj_idx==i)&(data1.condition=='compatible')].RT.mean()
    RT.iloc[i-101,2]=data1[(data1.subj_idx==i)&(data1.condition=='incompatible')].RT.mean()
    RT.iloc[i-101,3]=data1[(data1.subj_idx==i)&(data1.stim4=='imag')&(data1.condition=='compatible')].RT.mean()
    RT.iloc[i-101,4]=data1[(data1.subj_idx==i)&(data1.stim4=='imag')&(data1.condition=='incompatible')].RT.mean()
    RT.iloc[i-101,5]=data1[(data1.subj_idx==i)&(data1.stim4=='word')&(data1.condition=='compatible')].RT.mean()
    RT.iloc[i-101,6]=data1[(data1.subj_idx==i)&(data1.stim4=='word')&(data1.condition=='incompatible')].RT.mean()
    

RT=RT[(RT['subj_idx']!=106)&(RT['subj_idx']!=152)]
RT

,subj_idx,total_com,total_incom,imag_com,imag_incom,word_com,word_incom
0,101,1728.04,2661.32,1560.22,NaN,1741.45,2328.94
1,102,1378.09,1548.31,1337.63,1531.86,1351.72,1593.31
2,103,1697.3,1679.84,1523,1447.61,1679.32,1903.76
3,104,1284.14,1246.4,1209.08,1209.93,1355.27,1290.44
4,105,1103,1191.3,1095.02,1071.25,1156.88,1313.47
6,107,1364.33,1226.54,1223.26,1320.42,1480.57,1301.89
7,108,1698.78,1974.6,1457.58,1715.86,1910.13,1983.12
8,109,1339.69,1417.16,1288.82,1581.96,1187.44,1558.61
9,110,1108.76,1405.22,1007.88,1422.5,1126.9,1302
10,111,1178.67,1551.42,1047.3,1278.25,1404.57,1951.17


In [ ]:
RT.mean()

subj_idx        130.551724
total_com      1279.717622
total_incom    1456.040393
imag_com       1195.268107
imag_incom     1433.889103
word_com       1359.054307
word_incom     1572.948386
dtype: float64

In [ ]:
RT.std()

subj_idx        17.235390
total_com      252.437652
total_incom    347.211101
imag_com       218.225259
imag_incom     416.153824
word_com       276.710953
word_incom     362.102659
dtype: float64

In [ ]:
print(stats.ttest_rel(RT.total_com,RT.total_incom))


print(stats.ttest_rel(RT.loc[:,['imag_com','imag_incom']].dropna().imag_com,RT.loc[:,['imag_com','imag_incom']].dropna().imag_incom))
print(stats.ttest_rel(RT.loc[:,['word_com','word_incom']].dropna().word_com,RT.loc[:,['word_com','word_incom']].dropna().word_incom))


Ttest_relResult(statistic=-5.962752949885333, pvalue=1.6552008164348018e-07)
Ttest_relResult(statistic=-4.744242877454274, pvalue=1.6222627760507885e-05)
Ttest_relResult(statistic=-6.945875697154385, pvalue=3.915873537022637e-09)


In [ ]:
MAD =pd.DataFrame(columns=('subj_idx','total_com','total_incom','imag_com','imag_incom','word_com','word_incom'))
MAD.subj_idx=range(101,161)
for i in range(101,161):
    
    MAD.iloc[i-101,1]=data1[(data1.subj_idx==i)&(data1.condition=='compatible')].MAD.mean()
    MAD.iloc[i-101,2]=data1[(data1.subj_idx==i)&(data1.condition=='incompatible')].MAD.mean()
    MAD.iloc[i-101,3]=data1[(data1.subj_idx==i)&(data1.stim4=='imag')&(data1.condition=='compatible')].MAD.mean()
    MAD.iloc[i-101,4]=data1[(data1.subj_idx==i)&(data1.stim4=='imag')&(data1.condition=='incompatible')].MAD.mean()
    MAD.iloc[i-101,5]=data1[(data1.subj_idx==i)&(data1.stim4=='word')&(data1.condition=='compatible')].MAD.mean()
    MAD.iloc[i-101,6]=data1[(data1.subj_idx==i)&(data1.stim4=='word')&(data1.condition=='incompatible')].MAD.mean()
    

MAD=MAD[(MAD['subj_idx']!=106)&(MAD['subj_idx']!=152)]
MAD

,subj_idx,total_com,total_incom,imag_com,imag_incom,word_com,word_incom
0,101,0.295763,0.4935,0.184026,NaN,0.228725,0.234565
1,102,0.442288,0.552117,0.301607,0.373408,0.384041,0.533471
2,103,0.319498,0.338285,0.241109,0.208061,0.293696,0.25836
3,104,0.0214991,0.0754108,-0.0439271,0.0101741,0.0123979,0.0259383
4,105,0.352712,0.451692,0.219223,0.112551,0.330918,0.5091
6,107,0.616397,0.666115,0.444897,0.673924,0.769873,0.734227
7,108,0.38938,0.528094,0.253527,0.656133,0.464668,0.685996
8,109,0.297511,0.297223,0.0976848,0.358945,0.280323,0.377371
9,110,0.519144,0.892597,0.371398,0.776554,0.521262,0.791802
10,111,0.659168,0.670542,0.55213,0.450173,0.772565,0.782608


In [ ]:
print(stats.ttest_rel(MAD.total_com,MAD.total_incom))


print(stats.ttest_rel(MAD.loc[:,['imag_com','imag_incom']].dropna().imag_com,MAD.loc[:,['imag_com','imag_incom']].dropna().imag_incom))
print(stats.ttest_rel(MAD.loc[:,['word_com','word_incom']].dropna().word_com,MAD.loc[:,['word_com','word_incom']].dropna().word_incom))


Ttest_relResult(statistic=-3.4534264006895787, pvalue=0.001050687351100404)
Ttest_relResult(statistic=-3.6823627791151274, pvalue=0.0005427716939442737)
Ttest_relResult(statistic=0.011000301828324796, pvalue=0.9912616175576006)


In [ ]:
AD =pd.DataFrame(columns=('subj_idx','total_com','total_incom','imag_com','imag_incom','word_com','word_incom'))
AD.subj_idx=range(101,161)
for i in range(101,161):
    
    AD.iloc[i-101,1]=data1[(data1.subj_idx==i)&(data1.condition=='compatible')].AD.mean()
    AD.iloc[i-101,2]=data1[(data1.subj_idx==i)&(data1.condition=='incompatible')].AD.mean()
    AD.iloc[i-101,3]=data1[(data1.subj_idx==i)&(data1.stim4=='imag')&(data1.condition=='compatible')].AD.mean()
    AD.iloc[i-101,4]=data1[(data1.subj_idx==i)&(data1.stim4=='imag')&(data1.condition=='incompatible')].AD.mean()
    AD.iloc[i-101,5]=data1[(data1.subj_idx==i)&(data1.stim4=='word')&(data1.condition=='compatible')].AD.mean()
    AD.iloc[i-101,6]=data1[(data1.subj_idx==i)&(data1.stim4=='word')&(data1.condition=='incompatible')].AD.mean()
    

AD=AD[(AD['subj_idx']!=106)&(AD['subj_idx']!=152)]
AD

,subj_idx,total_com,total_incom,imag_com,imag_incom,word_com,word_incom
0,101,0.0678292,0.133111,0.0445865,NaN,0.0637534,0.0587929
1,102,0.119965,0.149696,0.076012,0.119226,0.105897,0.120747
2,103,0.0868453,0.1009,0.0767627,0.0871307,0.0724524,0.0767436
3,104,0.00738387,0.020532,-0.00751526,0.00455518,0.0024511,0.0111191
4,105,0.0746991,0.104966,0.0400476,0.0213523,0.0665725,0.128937
6,107,0.139101,0.135232,0.0623428,0.129386,0.201789,0.133358
7,108,0.138885,0.209301,0.0846867,0.227751,0.164352,0.308931
8,109,0.0726349,0.0630293,0.0357949,0.0769763,0.0547164,0.0825728
9,110,0.149091,0.258835,0.0974722,0.105083,0.156741,0.22246
10,111,0.194096,0.205401,0.151361,0.135655,0.242544,0.250262


In [ ]:
print(stats.ttest_rel(AD.total_com,AD.total_incom))


print(stats.ttest_rel(AD.loc[:,['imag_com','imag_incom']].dropna().imag_com,AD.loc[:,['imag_com','imag_incom']].dropna().imag_incom))
print(stats.ttest_rel(AD.loc[:,['word_com','word_incom']].dropna().word_com,AD.loc[:,['word_com','word_incom']].dropna().word_incom))


Ttest_relResult(statistic=-3.739532227076708, pvalue=0.0004295977017574891)
Ttest_relResult(statistic=-3.712931320138935, pvalue=0.0004933201426437914)
Ttest_relResult(statistic=-0.2636806359361745, pvalue=0.7929769585638945)


In [ ]:
AUC =pd.DataFrame(columns=('subj_idx','total_com','total_incom','imag_com','imag_incom','word_com','word_incom'))
AUC.subj_idx=range(101,161)
for i in range(101,161):
    
    AUC.iloc[i-101,1]=data1[(data1.subj_idx==i)&(data1.condition=='compatible')].AUC.mean()
    AUC.iloc[i-101,2]=data1[(data1.subj_idx==i)&(data1.condition=='incompatible')].AUC.mean()
    AUC.iloc[i-101,3]=data1[(data1.subj_idx==i)&(data1.stim4=='imag')&(data1.condition=='compatible')].AUC.mean()
    AUC.iloc[i-101,4]=data1[(data1.subj_idx==i)&(data1.stim4=='imag')&(data1.condition=='incompatible')].AUC.mean()
    AUC.iloc[i-101,5]=data1[(data1.subj_idx==i)&(data1.stim4=='word')&(data1.condition=='compatible')].AUC.mean()
    AUC.iloc[i-101,6]=data1[(data1.subj_idx==i)&(data1.stim4=='word')&(data1.condition=='incompatible')].AUC.mean()
    

AUC=AUC[(AUC['subj_idx']!=106)&(AUC['subj_idx']!=152)]
AUC

,subj_idx,total_com,total_incom,imag_com,imag_incom,word_com,word_incom
0,101,0.220054,0.309531,0.145692,NaN,0.153827,0.160095
1,102,0.291277,0.340261,0.202194,0.247365,0.3076,0.33155
2,103,0.177698,0.203211,0.186333,0.144136,0.136756,0.102896
3,104,0.0178585,0.05416,-0.0218207,0.0191663,0.00783935,0.0241843
4,105,0.267616,0.311087,0.176398,0.105972,0.249964,0.319403
6,107,0.400146,0.431087,0.299856,0.345778,0.463585,0.64272
7,108,0.274658,0.352167,0.199013,0.404061,0.307576,0.486553
8,109,0.190847,0.204441,0.0526703,0.246537,0.17709,0.305614
9,110,0.36653,0.481398,0.285871,0.0331475,0.355667,0.472297
10,111,0.410947,0.380599,0.413313,0.30114,0.429705,0.388627


In [ ]:
print(stats.ttest_rel(AUC.total_com,AUC.total_incom))


print(stats.ttest_rel(AUC.loc[:,['imag_com','imag_incom']].dropna().imag_com,AUC.loc[:,['imag_com','imag_incom']].dropna().imag_incom))
print(stats.ttest_rel(AUC.loc[:,['word_com','word_incom']].dropna().word_com,AUC.loc[:,['word_com','word_incom']].dropna().word_incom))

Ttest_relResult(statistic=-2.602391486231143, pvalue=0.01177775667762345)
Ttest_relResult(statistic=-1.7956323678810213, pvalue=0.0782547291739271)
Ttest_relResult(statistic=0.7723461475261373, pvalue=0.44310014328677994)


# EDA

In [ ]:
dataeda1=data1[['rt','MAD','AD','AUC']]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.tools as tls
import plotly.figure_factory as ff
sns.set(style="whitegrid")

In [ ]:
import seaborn as sns
sns.set()
sns.set_context(rc={"xtick.labelsize":26,'ytick.labelsize':26})
a1=sns.pairplot(dataeda1,height = 4, kind = "reg",corner=True,plot_kws={'scatter_kws':{'s':1}},diag_kws = {'alpha':1, 'bins':30})

plt.savefig('EDA-merge.jpg')


In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"
fig = ff.create_annotated_heatmap(dataeda1.corr().values.tolist(),
                                   
                                  y=dataeda1.columns.tolist(),
                                  x=dataeda1.columns.tolist(), 
                                  colorscale='Viridis',
                                  showscale=True
                                 )
fig.show()

In [ ]:
pingouin.rcorr(dataeda1,method='pearson',padjust='bonf',stars=True)

,rt,MAD,AD,AUC
rt,-,***,***,***
MAD,0.275,-,***,***
AD,0.323,0.892,-,***
AUC,0.158,0.864,0.779,-
